# **1 - Set up Kernel and Required Dependencies**

In [ ]:
!pip install torch torchdata transformers datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

**Load the datasets, Large Language Model (LLM), tokenizer, and configurator. Do not worry if you do not understand yet all of those components - they will be described and discussed later in the notebook.**

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

**In this use case, you will be generating a summary of a dialogue with the pre-trained Large Language Model (LLM) FLAN-T5 from Hugging Face. The list of available models in the Hugging Face transformers package can be found here.**

**Let's upload some simple dialogues from the DialogSum Hugging Face dataset. This dataset contains 10,000+ dialogues with the corresponding manually labeled summaries and topics.**

In [ ]:
from datasets import load_dataset
dataset = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

**Checking the dataset features**

In [ ]:
dataset['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 12460
})

In [ ]:
dataset['test']

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 1500
})

In [ ]:
dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [ ]:
dialogue = dataset['train'][0]['dialogue']
summary = dataset['train'][0]['summary']

print("Dialogue:", dialogue)
print("-------------------------------------------------------------------------------------------------")
print("Summary:", summary)


Dialogue: #Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.
----------------------------------------------------------

In [ ]:
example_indices = [10, 30]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
BASEL

**Load the FLAN-T5 model, creating an instance of the AutoModelForSeq2SeqLM class with the .from_pretrained() method.**

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**To perform encoding and decoding, you need to work with text in a tokenized form. Tokenization is the process of splitting texts into smaller units that can be processed by the LLM models.**

**Download the tokenizer for the FLAN-T5 model using AutoTokenizer.from_pretrained() method. Parameter use_fast switches on fast tokenizer. At this stage, there is no need to go into the details of that, but you can find the tokenizer parameters in the documentation.**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

**Testing the tokenizers whether it is performing well or not**

In [ ]:
sample_text_4_tokenization = "hey can you tell me the best place to play cricket in india"
tokenized_test = tokenizer(sample_text_4_tokenization,return_tensors="pt")
dotline='*'.join("" for i in range(100))
print('sample text')
print(dotline)
print(sample_text_4_tokenization,'\n')
tokenized_test['input_ids']
print('endoded sample text')
print(dotline,'\n')
print(tokenized_test['input_ids'])

sample text
***************************************************************************************************
hey can you tell me the best place to play cricket in india 

endoded sample text
*************************************************************************************************** 

tensor([[    3, 13133,    54,    25,   817,   140,     8,   200,   286,    12,
           577, 18096,    16, 18222,     1]])


# **Prompting**

**Now it's time to explore how well the base LLM summarizes a dialogue without any prompt engineering. Prompt engineering is an act of a human changing the prompt (input) to improve the response for a given task.**

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
BASELIN

**3 - Summarize Dialogue with an Instruction Prompt
Prompt engineering is an important concept in using foundation models for text generation. You can check out this blog from Amazon Science for a quick introduction to prompt engineering.**

3.1 - Zero Shot Inference with an Instruction Prompt
In order to instruct the model to perform a task - summarize a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called zero shot inference. You can check out this blog from AWS for a quick description of what zero shot learning is and why it is an important concept to the LLM model.

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

Summary:
    
----------------------------------------------------

**3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5**

Let's use a slightly different prompt. FLAN-T5 has many prompt templates that are published for certain tasks here. In the following code, you will use one of the pre-built FLAN-T5 prompts:

In [ ]:
example_indices=[12,45]
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: This Olympic park is so big!
#Person2#: Yes. Now we are in the Olympic stadium, the center of this park.
#Person1#: Splendid! When is it gonna be finished?
#Person2#: The whole stadium is to be finished this June.
#Person1#: How many seats are there in the stand?
#Person2#: Oh, there are 5000 seats in total.
#Person1#: I didn ' t know it would be so big!
#Person2#: It is! Look there, those are the tracks. And the jumping pit is over there.
#Person1#: Ah... I see. Hey, look the sign here, No climbing.
#Person2#: We put many signs with English translations for foreign visitors.

What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is surprised a

**4 - Summarize Dialogue with One Shot and Few Shot Inference**

One shot and few shot inference are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task. You can read more about it in this blog from HuggingFace.




4.1 - One Shot Inference
Let's build a function that takes a list of example_indices_full, generates a prompt with full examples, then at the end appends the prompt which you want the model to complete (example_index_to_summarize). You will use the same FLAN-T5 prompt template from section 3.2.

In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

In [ ]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

**Now pass this prompt to perform the one shot inference:**

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


# 4.2 - Few Shot Inference
Let's explore few shot inference by adding two more full dialogue-summary pairs to your prompt.

In [ ]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. 

Now pass this prompt to perform a few shot inference:

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


**5 - Generative Configuration Parameters for Inference
You can change the configuration parameters of the generate() method to see a different output from the LLM. So far the only parameter that you have been setting was max_new_tokens=50, which defines the maximum number of tokens to generate. A full list of available parameters can be found in the Hugging Face Generation documentation.**

A convenient way of organizing the configuration parameters is to use GenerationConfig class.

**Exercise:**

Change the configuration parameters to investigate their influence on the output.

Putting the parameter do_sample = True, you activate various decoding strategies which influence the next token from the probability distribution over the entire vocabulary. You can then adjust the outputs changing temperature and other parameters (such as top_k and top_p).

Uncomment the lines in the cell below and rerun the code. Try to analyze the results. You can read some comments below.

In [ ]:
generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



# **Using another Dataset**

**1- Loading the dataset**

**Dataset Summary**

The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

In [ ]:
from datasets import load_dataset

ds = load_dataset("abisee/cnn_dailymail", "1.0.0")

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Checking the features and the data

In [ ]:
# training data
ds['train']

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [ ]:
# test data
ds['test']

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

here we can take highlight as the summary

In [ ]:
article=ds['train'][0]['article']
highlight=ds['train'][0]['highlights']
print(article)
print(highlight)

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

**Tokenizer , LLM Model**

we have already loaded the LLM model and Tokenizer from the Hugging Face

**Without any prompts**

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = ds['test'][index]['article']
    summary = ds['test'][index]['highlights']
    inputs = tokenizer(article, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True
    )
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
(CNN)Seventy years ago, Anne Frank died of typhus in a Nazi concentration camp at the age of 15. Just two weeks after her supposed death on March 31, 1945, the Bergen-Belsen concentration camp where she had been imprisoned was liberated -- timing that showed how close the Jewish diarist had been to surviving the Holocaust. But new research released by the Anne Frank House shows that Anne and her older sister, Margot Frank, died at least a month earlier than previously thought. Researchers re-examined archives of the Red Cross, the International Training Service and the Bergen-Belsen Memorial, along with testimonies of survivors. They concluded that Anne and Margot probably did not survive to March 1945 -- contradicting the date of death which had previously been de

# **Using own data**

In [ ]:
import pandas as pd

# Sample data with longer dialogues and their corresponding summaries
data = {
    'dialogue': [
        "Hi there! How has your day been so far? I had a pretty busy morning, but it has been good. I was just catching up on some work and trying to get a few tasks done before lunch. How about you? What have you been up to today?",
        "I'm feeling overwhelmed with all the work I have to do this week. Between meetings, emails, and project deadlines, there's just so much on my plate. I haven't had time to take a break or even think straight. It's going to be a very stressful week, I can already tell.",
        "Wow, the weather outside is just perfect today! The sun is shining, there's a light breeze, and it feels like the perfect temperature for a walk. I can't remember the last time I got outside for a bit. I think I might head to the park later to enjoy the fresh air and clear my head.",
        "Did you catch the game last night? It was such an intense match! The teams were neck and neck the entire time, and I couldn't believe how it ended. I was on the edge of my seat the whole time. It was one of those games you can't stop thinking about because it was just so thrilling.",
        "Have you heard about the new restaurant that opened downtown? It's been getting a lot of buzz, so I decided to check it out last night. The ambiance was fantastic, and the food was absolutely delicious. I tried the signature dish, and it was even better than I expected. You should definitely go try it sometime!"
    ],
    'summary': [
        "Person A asks how Person B is doing and shares about their busy morning, while Person B reciprocates the question.",
        "Person A expresses feeling overwhelmed by work and mentions how busy their week is with deadlines and meetings.",
        "Person A talks about the nice weather and plans to take a walk in the park to enjoy the outdoors.",
        "Person A describes the excitement of watching an intense game the previous night and shares how thrilling it was.",
        "Person A recommends a new restaurant downtown, describing the great food and ambiance."
    ]
}

# Creating a DataFrame
df = pd.DataFrame(data)


Displaying the data

In [ ]:
df

,dialogue,summary
0,Hi there! How has your day been so far? I had ...,Person A asks how Person B is doing and shares...
1,I'm feeling overwhelmed with all the work I ha...,Person A expresses feeling overwhelmed by work...
2,"Wow, the weather outside is just perfect today...",Person A talks about the nice weather and plan...
3,Did you catch the game last night? It was such...,Person A describes the excitement of watching ...
4,Have you heard about the new restaurant that o...,"Person A recommends a new restaurant downtown,..."


# Zero shot prompting

**Zero-shot, One-shot, and Few-shot Prompts in NLP
These terms refer to the number of examples provided to a model (typically in a prompt) to help it perform a specific task. Here's a summary of each type:**

1. Zero-shot Prompt
Definition: A zero-shot prompt involves asking the model to perform a task without providing any examples.
Key Idea: The model is expected to generalize and perform the task based on its pre-existing knowledge.
Use Case: It’s useful when you have no task-specific data or when you want to test the model's ability to handle tasks it has not seen before.

Example:

Prompt: "Translate the following English text to French: 'Hello, how are you?'"

Output: "Bonjour, comment ça va ?"

Explanation: The model doesn’t receive any prior examples of English-to-French translations. It relies on the pre-trained knowledge to generate the translation.

In [ ]:
dialogue=df.iloc[0,0]
summary=df.iloc[1]
promt= f"""
summarize : {dialogue}
"""
inputs = tokenizer(promt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)
print(dash_line)
print(f'INPUT PROMPT:\n{promt}')
print(dash_line)
print(f'Actual HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')



---------------------------------------------------------------------------------------------------
INPUT PROMPT:

summarize : Hi there! How has your day been so far? I had a pretty busy morning, but it has been good. I was just catching up on some work and trying to get a few tasks done before lunch. How about you? What have you been up to today?

---------------------------------------------------------------------------------------------------
Actual HUMAN SUMMARY:
dialogue    I'm feeling overwhelmed with all the work I ha...
summary     Person A expresses feeling overwhelmed by work...
Name: 1, dtype: object
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
I'm having a good day.



# One - shot prompt

**One-shot Prompt**

Definition: A one-shot prompt provides a single example of the task.
Key Idea: The model gets a single example and is then expected to perform the task for a new, unseen instance.
Use Case: It’s useful when you have very few examples and want to teach the model the task using just one instance.

Example:

Prompt: "Translate the following English text to French: 'Hello, how are you?'
 Here is an example: 'I am happy' -> 'Je suis heureux.' Now, translate: 'I am sad.'"

Output: "Je suis triste."

Explanation: The model is given just one example of a translation and then asked to translate a new sentence.

In [ ]:
#promt for LLMModel
dialogue=df.iloc[0,0]
summary=df.iloc[0,1]
one_shot_prompt=f"""
dialogue : {dialogue}
summary of the dialogue is : {summary}
"""
test_dialogue=df.iloc[2,0]
one_shot_prompt += f"""
dialogue : {test_dialogue}
the summary of the dialogue is :
"""

In [ ]:
one_shot_prompt

"\ndialogue : Hi there! How has your day been so far? I had a pretty busy morning, but it has been good. I was just catching up on some work and trying to get a few tasks done before lunch. How about you? What have you been up to today?\nsummary of the dialogue is : Person A asks how Person B is doing and shares about their busy morning, while Person B reciprocates the question.\n\ndialogue : Wow, the weather outside is just perfect today! The sun is shining, there's a light breeze, and it feels like the perfect temperature for a walk. I can't remember the last time I got outside for a bit. I think I might head to the park later to enjoy the fresh air and clear my head.\nthe summary of the dialogue is :\n"

In [ ]:
summary = df.iloc[2,1]
inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Person A talks about the nice weather and plans to take a walk in the park to enjoy the outdoors.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
The weather is perfect today.


# Few shot prompt

**Few-shot Prompt**

Definition: A few-shot prompt provides a small number of examples (more than one but typically less than five).
Key Idea: The model is given a few examples of the task to learn from and then performs the task for new instances.
Use Case: It’s useful when you have a limited number of labeled examples to help the model understand the task context.
Example:
Prompt: "Translate the following English text to French:
'I am happy' -> 'Je suis heureux.'
'I am sad' -> 'Je suis triste.'
'I am excited' -> 'Je suis excité.' Now, translate: 'I am angry.'"
Output: "Je suis en colère."
Explanation: The model is given three examples of translations and is asked to generate a translation for a new sentence, "I am angry."

In [ ]:
dialogue_1=df.iloc[0,0]
summary_1=df.iloc[0,1]
dialogue_2=df.iloc[1,0]
summary_2=df.iloc[1,1]
dialogue_3=df.iloc[2,0]
summary_3=df.iloc[2,1]

few_shot_prompt=f"""
dialogue : {dialogue_1}
summary of the {dialogue_1} is : {summary_1}
dialogue : {dialogue_2}
summary of the {dialogue_2} is : {summary_2}
dialogue : {dialogue_3}
give summary of the {dialogue_3} :
"""


In [ ]:
few_shot_prompt

"\ndialogue : Hi there! How has your day been so far? I had a pretty busy morning, but it has been good. I was just catching up on some work and trying to get a few tasks done before lunch. How about you? What have you been up to today?\nsummary of the Hi there! How has your day been so far? I had a pretty busy morning, but it has been good. I was just catching up on some work and trying to get a few tasks done before lunch. How about you? What have you been up to today? is : Person A asks how Person B is doing and shares about their busy morning, while Person B reciprocates the question.\ndialogue : I'm feeling overwhelmed with all the work I have to do this week. Between meetings, emails, and project deadlines, there's just so much on my plate. I haven't had time to take a break or even think straight. It's going to be a very stressful week, I can already tell.\nsummary of the I'm feeling overwhelmed with all the work I have to do this week. Between meetings, emails, and project dead

In [ ]:
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary_3}\n')
print(dash_line)
print(f'MODEL GENERATION - Few SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Person A talks about the nice weather and plans to take a walk in the park to enjoy the outdoors.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
summary of the Wow, the weather outside is just perfect today! The sun is shining, there's a light breeze, and it feels like the perfect temperature for a walk. I can't remember the last time I got outside for
